<a href="https://colab.research.google.com/github/ace-racer/Extending-Board-Games-using-deep-learning/blob/master/chess_piece_detection/app/Balanced_Siamese_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
Initial code from: https://sorenbouma.github.io/blog/oneshot/

"""
import keras
from keras.layers import Input, Conv2D, Lambda, average, Dense, Flatten,MaxPooling2D, BatchNormalization, Dropout, Activation, Subtract, subtract
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (100, 100, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)

#build convnet to use in each siamese 'leg'
convnet = Sequential()

convnet.add(Conv2D(32,(5,5),input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Activation('relu'))
convnet.add(MaxPooling2D())

convnet.add(Conv2D(64,(4,4), kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(BatchNormalization())
convnet.add(Activation('relu'))
convnet.add(MaxPooling2D())

convnet.add(Conv2D(128,(4,4), kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(BatchNormalization())
convnet.add(Activation('relu'))
convnet.add(Flatten())
convnet.add(Dropout(0.4))
convnet.add(Dense(1024,activation="relu",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

#merge two encoded inputs with the average
both = subtract([encoded_l,encoded_r])
both = K.abs(both)
# both = Dense(256, activation='relu')(both)
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(both)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)


optimizer = Adam(0.0005)

siamese_net.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

print(siamese_net.count_params())
print(siamese_net.summary())

In [0]:
! git clone https://github.com/ace-racer/Chess-Pieces-Data.git
! ls

In [0]:
import numpy as np
import keras
import os
import itertools
import random
from collections import Counter, defaultdict
from itertools import product, combinations
import math

import cv2
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

#samples_per_type = {"b": 30, "n": 25, "k": 25, "p": 35, "q": 25, "r": 35}
samples_per_type = {"b": 3, "n": 2, "k": 2, "p": 3, "q": 2, "r": 3}

# training parameters
IMAGE_SIZE = (100, 100)
CHECKPOINTS_LOCATION = "weights"
LOGS_LOCATION = "logs"
BATCH_SIZE = 32
NUM_EPOCHS = 50

# change as required
IMAGES_LOCATION = "Chess-Pieces-Data/crawled_1901/"
#IMAGES_LOCATION = "H:\\AR-ExtendingOnlineGames\\crawled_chess_piece_images"

if not os.path.exists(CHECKPOINTS_LOCATION):
    os.makedirs(CHECKPOINTS_LOCATION)

if not os.path.exists(LOGS_LOCATION):
    os.makedirs(LOGS_LOCATION)

def generate_paired_instances_by_ratio(folder_name, total_instances = 6000, different_records_ratio = 0.5):
    
    data = []
    label_values = []
    for type_name in samples_per_type:
        piece_type_folder = os.path.join(folder_name, type_name)
        for f in (os.listdir(piece_type_folder)):
            img_file_loc = os.path.join(piece_type_folder, f)
            data.append(img_file_loc)
            label_values.append(type_name)
    
    num_categories = 6

    # Get the counts of the individual labels
    label_counts = Counter(label_values)
    
    # Get the label indices in the original data read from the file
    label_indices = defaultdict(list)
    for itr, val in enumerate(label_values):
        label_indices[val].append(itr)
    
    num_same_items_per_category = int(math.ceil(np.sqrt((( 1- different_records_ratio ) * total_instances) / num_categories)))
    num_different_items_per_category = int(math.ceil(np.sqrt((2 * different_records_ratio * total_instances)/(num_categories * (num_categories - 1)))))
    print("Num same items per category: " + str(num_same_items_per_category))
    print("Num different items per category: " + str(num_different_items_per_category))

    most_common_categories = [x for x, _ in label_counts.most_common(num_categories)]
    print("Most common categories...")
    print(most_common_categories)
    
    pairwise_indices_same_items = []
    for label in most_common_categories:
        required_indices = label_indices[label][:num_same_items_per_category]
        similar_item_index_pairs = list(product(required_indices, required_indices))
        pairwise_indices_same_items.extend(similar_item_index_pairs)

    pairwise_indices_different_items = []
    category_pairs = combinations(most_common_categories, 2)

    for cat1, cat2 in category_pairs:
        category1_indices = label_indices[cat1][:num_different_items_per_category]
        category2_indices = label_indices[cat2][:num_different_items_per_category]
        different_items_index_pairs = list(product(category1_indices, category2_indices))
        pairwise_indices_different_items.extend(different_items_index_pairs)

    print("Num same category pairs: " + str(len(pairwise_indices_same_items)))
    print("Num different category pairs: " + str(len(pairwise_indices_different_items)))

    instances_with_labels = []
    for idx1, idx2 in pairwise_indices_same_items:
        label = int(label_values[idx1] == label_values[idx2])
        
        img1 = cv2.imread(data[idx1])
        img1 = cv2.resize(img1, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

        img2 = cv2.imread(data[idx2])
        img2 = cv2.resize(img2, IMAGE_SIZE, interpolation=cv2.INTER_AREA)
        
        instances_with_labels.append((img1, img2, label))

    for idx1, idx2 in pairwise_indices_different_items:
        label = int(label_values[idx1] == label_values[idx2])

        img1 = cv2.imread(data[idx1])
        img1 = cv2.resize(img1, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

        img2 = cv2.imread(data[idx2])
        img2 = cv2.resize(img2, IMAGE_SIZE, interpolation=cv2.INTER_AREA)
        
        instances_with_labels.append((img1, img2, label))

    random.shuffle(instances_with_labels)
    instances = np.array([[x[0], x[1]] for x in instances_with_labels])
    labels = np.array([x[2] for x in instances_with_labels])

    return instances, labels

X_train_original = []
y_train_original = []


training_images = os.path.join(IMAGES_LOCATION, "train")

X_train_original, y_train_original = generate_paired_instances_by_ratio(training_images)

X_train_original = np.array(X_train_original)
X_train_original = X_train_original.astype('float32')
X_train_original /= 255
y_train_original = np.array(y_train_original)

print(X_train_original.shape)
print(y_train_original.shape)

# split into train and validation splits
X_train, X_test, y_train, y_test = train_test_split(X_train_original, y_train_original, test_size=0.25, random_state=42, stratify = y_train_original)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train_left = X_train[:, 0, ...]
X_train_right = X_train[:, 1, ...]
print(X_train_left.shape)
print(X_train_right.shape)

X_test_left = X_test[:, 0, ...]
X_test_right = X_test[:, 1, ...]


filepath = os.path.join(CHECKPOINTS_LOCATION, "siamese.hdf5")

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

earlystop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=10, verbose=1, mode='max')

tensorboard = TensorBoard(log_dir=LOGS_LOCATION, histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, earlystop, tensorboard]


model = siamese_net
X_train_instances = [X_train_left, X_train_right]
X_test_instances = [X_test_left, X_test_right]

hist = model.fit(X_train_instances, y_train, shuffle=True, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1, validation_data=(X_test_instances, y_test), callbacks=callbacks_list)




Num same items per category: 23
Num different items per category: 15
Most common categories...
['r', 'b', 'n', 'k', 'p', 'q']
Num same category pairs: 3174
Num different category pairs: 3375
(6549, 2, 100, 100, 3)
(6549,)
(4911, 2, 100, 100, 3)
(1638, 2, 100, 100, 3)
(4911,)
(1638,)
(4911, 100, 100, 3)
(4911, 100, 100, 3)
Train on 4911 samples, validate on 1638 samples
Epoch 1/50
4911/4911 [==============================] - 842s 172ms/step - loss: 9.7205 - acc: 0.5437 - val_loss: 7.4238 - val_acc: 0.5812

Epoch 00001: val_acc improved from -inf to 0.58120, saving model to weights/siamese.hdf5
Epoch 2/50
4911/4911 [==============================] - 842s 171ms/step - loss: 3.3655 - acc: 0.6856 - val_loss: 2.4357 - val_acc: 0.7692

Epoch 00002: val_acc improved from 0.58120 to 0.76923, saving model to weights/siamese.hdf5
Epoch 3/50
4911/4911 [==============================] - 839s 171ms/step - loss: 2.1049 - acc: 0.7502 - val_loss: 1.7886 - val_acc: 0.7637

Epoch 00003: val_acc did not i

KeyboardInterrupt: ignored

In [0]:
!ls

Chess-Pieces-Data  logs  sample_data  weights


In [0]:
from google.colab import files

!zip -r weights.zip weights
!zip -r logs.zip logs



  adding: weights/ (stored 0%)
  adding: weights/siamese.hdf5 (deflated 6%)
  adding: logs/ (stored 0%)
  adding: logs/events.out.tfevents.1548678183.1ef9e7bc7e09 (deflated 8%)
  adding: logs/events.out.tfevents.1548677857.1ef9e7bc7e09 (deflated 8%)
  adding: logs/events.out.tfevents.1548678453.1ef9e7bc7e09 (deflated 8%)
  adding: logs/events.out.tfevents.1548679544.1ef9e7bc7e09 (deflated 8%)


In [0]:
!ls

Chess-Pieces-Data  logs  logs.zip  sample_data	weights  weights.zip


In [0]:
files.download("weights.zip")

In [0]:
files.download("logs.zip")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 60082, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil